# <font color='blue'> Monty Hall Problem

In [1]:
from IPython.core.interactiveshell import InteractiveShell # 모든 명령어 출력
InteractiveShell.ast_node_interactivity = 'all'
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 2.1. Problem Description:
#### In search of a new car, the player picks a door, say 1. The game host then opens one of the other doors, say 3, to reveal a goat and offers to let the player switch from door 1 to door 2.
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Monty_open_door.svg/330px-Monty_open_door.svg.png'>
https://en.wikipedia.org/wiki/Monty_Hall_problem

In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

# Defining the network structure
model = BayesianNetwork([("C", "H"), ("P", "H")])

# Defining the CPDs:
cpd_c = TabularCPD("C", 3, [[0.33], [0.33], [0.33]])
cpd_p = TabularCPD("P", 3, [[0.33], [0.33], [0.33]])
cpd_h = TabularCPD(
    "H",
    3,
    [
        [0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
        [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
        [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0],
    ],
    evidence=["C", "P"],
    evidence_card=[3, 3],
)

# Associating the CPDs with the network structure.
model.add_cpds(cpd_c, cpd_p, cpd_h)

# Some other methods
model.get_cpds()

[<TabularCPD representing P(C:3) at 0x1d91833c220>,
 <TabularCPD representing P(P:3) at 0x1d9182e8850>,
 <TabularCPD representing P(H:3 | C:3, P:3) at 0x1d9182e5460>]

<img src='https://pgmpy.org/_images/examples_Monty_Hall_Problem_3_0.png' width=400 height=400>



P=prize(new car), C=contestor, H= true

In [3]:
c = ['door_1']*3 + ['door_2']*3 + ['door_3']*3
p = ['door_1', 'door_2', 'door_3']*3

data = pd.DataFrame(cpd_h.get_values(), 
             index= ['Host=1', 'Host=2', 'Host=3'],
             columns = [c, p])
data.index.name ='사회자(Host)가 아래 문을 열 확률'
data

door_1               door_2               door_3         \
                      door_1 door_2 door_3 door_1 door_2 door_3 door_1 door_2   
사회자(Host)가 아래 문을 열 확률                                                           
Host=1                   0.0    0.0    0.0    0.0    0.5    1.0    0.0    1.0   
Host=2                   0.5    0.0    1.0    0.0    0.0    0.0    1.0    0.0   
Host=3                   0.5    1.0    0.0    1.0    0.5    0.0    0.0    0.0   

                              
                      door_3  
사회자(Host)가 아래 문을 열 확률         
Host=1                   0.5  
Host=2                   0.5  
Host=3                   0.0

### 맨 윗줄 도전자가 '1'번 선택했는데 신차가 '1'번에 있으면 '1'번을 열 확률은 '0', '2'번 혹은 '3'번을 열 확률은 각각 '0.5'  --> 표 첫번째 컬럼 (0.0, 0.5, 0.5)
### 맨 윗줄 도전자가 '1'번 선택했는데 신차가 '2'번에 있으면 '1'번을 열 확률은 '0', '2'번은 신차가 있으니 열수가 없고 '3'번을 열 확률은 '1.0'  --> 표 두번째 컬럼 (0.0, 0.0, 1.0)

In [4]:
# check_model check for the model structure and the associated CPD and returns True if everything is correct otherwise throws an exception
model.check_model()

True

## 도전자가 '0'번 문을 열고 사회자가 '2'번문을 열면 도전자가 '1'으로 선택을 변경하면 상금을 탈 사후확률이 66.77%로 증가하게됨
## 변경하지 않으면 33.33%

In [5]:
# Infering the posterior probability
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior_p = infer.query(["P"], evidence={"C": 0, "H": 2})
print(posterior_p)

+------+----------+
| P    |   phi(P) |
+======+==========+
| P(0) |   0.3333 |
+------+----------+
| P(1) |   0.6667 |
+------+----------+
| P(2) |   0.0000 |
+------+----------+


# End